In [1]:
import pandas as pd

In [265]:
df = pd.read_csv('data/2018_transactions_original.csv',
                 parse_dates=['ARRIVAL', 'DEPART', 'BOOKING'],
                 dayfirst=True)
df_units = pd.read_csv('data/room_number_type_mapping.csv',
                       dtype={'room_number': object})

In [266]:
df = df.drop_duplicates()

In [267]:
# convert units to codes
df['UNIT'] = df['UNIT'].astype(str)
room_mapping_df = df_units.set_index('room_number')
room_mapping_dict = room_mapping_df.to_dict()['room_type']
df['UNIT_num'] = df['UNIT'].copy()
df['UNIT'] = df['UNIT'].replace(room_mapping_dict)
df['UNIT_num'] = df['UNIT_num'].astype(int)

In [268]:
# create group ID's and group sizes
df['group_id'] = df.groupby(['UNIT_num', 'ARRIVAL', 'DEPART']).ngroup()
group_sizes = df.groupby('group_id').size()
df = df.merge(pd.DataFrame(group_sizes), on='group_id', how='outer')
df = df.rename(index=str, columns={0:'grp_size'})
df = df.set_index('group_id')
df = df.drop(['UNIT_num'], axis=1)

In [271]:
# final sorting by booking date and resno
df = df.sort_values(['BOOKING', 'RESNO'])

In [278]:
df.to_csv('data/2018_preprocessed.csv')